This is currently the best code for calibrating the tagger timing and saving the offsets.

# A (Possible) Code for Calibrating the Tagger Timing

First, we import the tools we'll need:

In [1]:
import Acqu as aq
import AcquDetector as aqdet
import ROOT
import numpy as np
import json
from collections import OrderedDict
from rootpy.plotting import histogram, Hist2D, Hist, Canvas

Welcome to JupyROOT 6.16/00


Import the data to be analyzed:

In [2]:
inFile = '/scratch/2019-05_Timepix/Timepix_33.dat'
aq.openFile(inFile)

Mk2 Data


Load in the detector file for the Tagger:

In [3]:
aqdet.LoadDetectors(['taggerNewer.json'])

taggerNewer.json


We set up histograms for each of the channels, and then fill them with tagger times from our data:

In [4]:
taggerChannels = aqdet.Channels['Tagger']     # number of channels in the tagger (368)

histos = [None]*taggerChannels                # set up an array of 368 histograms to fill      

for i in range(taggerChannels):               # for each tagger channel
    histos[i] = Hist(1300,-500,800)           # set up a histogram for that tagger channel

def plotCalTagger():
    data = aqdet.Calibrate(aq.adcArray)              # calibrate the data
    taggerTimes = aqdet.Arrays['Tagger']['Time']     # get the tagger times
    
    for dat in taggerTimes:                          # for each data point
        for time in dat['Time']:                     # for each tagger time
            for i in range(taggerChannels):          # for each channel
                if (dat['channel']==i):              # if the channel of that data point is the current channel
                    histos[i].Fill(time)             # then put that time in the histogram
                    
    if(aq.eventNo%5000==0):                          # print a processing statement every 5000 events
        print("Events Processed: ",aq.eventNo)
    
aq.runFunction(plotCalTagger,0,500000)               # run this process over 500000 events
print("end")

('Events Processed: ', 5000)
('Events Processed: ', 10000)
('Events Processed: ', 15000)
('Events Processed: ', 20000)
('Events Processed: ', 25000)
('Events Processed: ', 30000)
('Events Processed: ', 35000)
('Events Processed: ', 40000)
('Events Processed: ', 45000)
('Events Processed: ', 50000)
('Events Processed: ', 55000)
('Events Processed: ', 60000)
('Events Processed: ', 65000)
('Events Processed: ', 70000)
('Events Processed: ', 75000)
('Events Processed: ', 80000)
('Events Processed: ', 85000)
('Events Processed: ', 90000)
('Events Processed: ', 95000)
('Events Processed: ', 100000)
('Events Processed: ', 105000)
('Events Processed: ', 110000)
('Events Processed: ', 115000)
('Events Processed: ', 120000)
('Events Processed: ', 125000)
('Events Processed: ', 130000)
('Events Processed: ', 135000)
('Events Processed: ', 140000)
('Events Processed: ', 145000)
('Events Processed: ', 150000)
('Events Processed: ', 155000)
('Events Processed: ', 160000)
('Events Processed: ', 16500

('Events Processed: ', 1335000)
('Events Processed: ', 1340000)
('Events Processed: ', 1345000)
('Events Processed: ', 1350000)
('Events Processed: ', 1355000)
('Events Processed: ', 1360000)
('Events Processed: ', 1365000)
('Events Processed: ', 1370000)
('Events Processed: ', 1375000)
('Events Processed: ', 1380000)
('Events Processed: ', 1385000)
('Events Processed: ', 1390000)
('Events Processed: ', 1395000)
('Events Processed: ', 1400000)
('Events Processed: ', 1405000)
('Events Processed: ', 1410000)
('Events Processed: ', 1415000)
('Events Processed: ', 1420000)
('Events Processed: ', 1425000)
('Events Processed: ', 1430000)
('Events Processed: ', 1435000)
('Events Processed: ', 1440000)
('Events Processed: ', 1445000)
('Events Processed: ', 1450000)
('Events Processed: ', 1455000)
('Events Processed: ', 1460000)
('Events Processed: ', 1465000)
('Events Processed: ', 1470000)
('Events Processed: ', 1475000)
('Events Processed: ', 1480000)
('Events Processed: ', 1485000)
('Events

Now we plot the data for the tagger channels and apply a gaussian fit (with an offset) to them:

In [5]:
ROOT.enableJSVis()

tagger_times = ROOT.TFile("tagger_times.root","RECREATE")     # create a root file to put all the histograms into

myFit = ROOT.TF1("myFit","gaus(0)+pol0(3)")                   # make a gaussian fit with an offset/background
myFit.SetParNames("constant","mean","stdev","background")     # label the parameters

empties = []      # set up an array to remember the empty or near empty histograms

con = [None]*taggerChannels           # set up an array for the constant value
peak = [None]*taggerChannels          # set up an array for the peak locations
sigma = [None]*taggerChannels         # set up an array for the standard deviations of the peaks
bgLevel = [None]*taggerChannels       # set up an array for the background levels

sig = 1.1    # initialize a standard deviation value to try
bg = 100     # initialize a background value to try

for i in range(taggerChannels):              # for each channel in the tagger                                   
    histos[i].SetTitle("Channel "+str(i))    # set the title for the histogram              
    
    if(histos[i].GetEntries()>10):                         # if the histogram has entries
        binmax = histos[i].GetMaximumBin()                 # find the maximum number of entries in a bin
        pk = histos[i].GetXaxis().GetBinCenter(binmax)     # find the bin corresponding to the max
        
        myFit.SetParameter(1,pk)          # try a peak value near the fullest bin
        myFit.SetParameter(2,sig)         # try this standard deviation value for the fit       
        myFit.SetParameter(3,bg)          # try this background value for the fit
        
        histos[i].Fit('myFit','SQ','',-300,600)    # fit the histogram with a gaussian
        
        peak[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(1)           # save the peak position
        sigma[i] = abs(histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(2))     # save the standard deviation
        bgLevel[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(3)        # save the background level
        con[i] = histos[i].Fit('myFit','SQ','',-300,600).Get().Parameter(0) 
 
        if(ROOT.gMinuit.fCstatu!="CONVERGED " or not(120<peak[i]<160) or not(-2<sigma[i]<4) or bgLevel[i]<0 or con[i]<0):  
            print("Channel "+str(i)+" fit has failed.")      # if this is printed, see OldTaggerCalib.ipynb for possible while loop
            
        
        sig = sigma[i]       # try using this sigma value for the next channel
        bg = bgLevel[i]      # try using this background value for the next channel
       
    else:       # if the histogram is empty                                     
        print("Histogram "+str(i)+" is empty (has "+str(histos[i].GetEntries())+" entries).")    # then say so 
        empties.append(i)               # note that this channel is an empty histogram
     
    histos[i].Write("tagger_times")     # write the histogram to a root file for viewing

INFO:ROOT.TCanvas.MakeDefCanvas]  created default TCanvas with name c1


Histogram 216 is empty (has 3.0 entries).
Histogram 273 is empty (has 1.0 entries).
Histogram 327 is empty (has 3.0 entries).


Finding how much each peak is off from zero, and setting it back to zero (likely not needed - but a good visual tool):

In [42]:
tagger_calib = ROOT.TFile("tagger_calib.root","RECREATE")    # create a root file to put all the histograms into

histos_calib = [None]*taggerChannels           # set up an array of 368 histograms to fill

for i in range(taggerChannels):                # for each channel in the tagger
    histos_calib[i] = Hist(1300,-500,800)      # create a histogram

for i in range(taggerChannels):         # for each channel in the tagger             
    if (peak[i]!=None):                 # so long as there is a peak value saved
        histos_calib[i] = Hist(1300,int(-500-peak[i]),int(800-peak[i]))     # create a histogram shifted back by the value of the peak      
        for j in range(1300):                                               # for each bin in that channel's data
            histos_calib[i].SetBinContent(j,histos[i].GetBinContent(j))     # give the bin that channel's data, now shifted back by the peak value
     
    histos_calib[i].SetTitle("Channel "+str(i))     # set the title for the histogram
    
    histos_calib[i].Write("tagger_calib")           # write the histogram to a root file for viewing

Next, we must import the json file, change the channel offset values, and then save the array again as a new json file. Note that the empty histograms write a value of 'None' to the json file for their offsets.

In [43]:
calibData = json.load(open('taggerNewer.json'), object_pairs_hook=OrderedDict)     # load the tagger calibration data and keep its original ordering

i = 0                                   # starting at channel 0
for chan in calibData['channels']:      # for each of the tagger channels
    if(i not in empties):               # so long as the histogram has data fitted
        chan['offset'][0] += peak[i]/chan['scale'][0]      # change the offset value to the correct one
    i+=1                                # move on to the next channel

json.dump(calibData, open('taggerWithOffsets.json', 'w'), indent = 2)              # write the changed data to a new json file